In [1]:
import sys
import json
import re
import pandas as pd
from collections import defaultdict, Counter
from math import log
from random import shuffle
import csv
import multiprocessing
from multiprocessing import Pool
import numpy as np
import pickle

In [2]:
time_start = pd.Timestamp('2020-01-12')
time_end = pd.Timestamp('2020-07-13')

Analysis


In [7]:
# df_test = pd.read_parquet('/home/wtchiang/ADS_crossDomainRec/raw_data/interaction_log_v6_20200724_test.parquet', engine='pyarrow')
df_test_click = pd.read_parquet('/home/syliu/AIR/Data/kk_Data/different_interaction_test_data/interaction_log_v6_20210426_test_click.parquet', engine='pyarrow')
df_test_download = pd.read_parquet('/home/syliu/AIR/Data/kk_Data/different_interaction_test_data/interaction_log_v6_20210426_test_download.parquet', engine='pyarrow')
df_test_favor = pd.read_parquet('/home/syliu/AIR/Data/kk_Data/different_interaction_test_data/interaction_log_v6_20210426_test_favor.parquet', engine='pyarrow')
df_test_purchase = pd.read_parquet('/home/syliu/AIR/Data/kk_Data/different_interaction_test_data/interaction_log_v6_20210426_test_purchase.parquet', engine='pyarrow')
df_test_search = pd.read_parquet('/home/syliu/AIR/Data/kk_Data/different_interaction_test_data/interaction_log_v6_20210426_test_search.parquet', engine='pyarrow')


In [ ]:
df = pd.read_parquet('/home/wtchiang/ADS_crossDomainRec/raw_data/interaction_log_v6_20200724_train.parquet', engine='pyarrow')
# df = df[ (df['client_upload_timestamp'] > time_start) & (df['client_upload_timestamp'] <= time_end) ]
## pyarrow is for supporting Apache Parquet Format

In [4]:
df

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
0,01640,1TwKybXi09qknpFn3DClPOOVCAK,movie,44,2020-03-17 10:34:33,2020-03-17 10:34:36,play
1,02756,1TwKyRGvhmV61mO2uokTMFgNBup,movie,97,2020-03-04 06:50:48,2020-03-04 06:50:51,play
2,09485,1TwKjd28kvL8VC151WbaKcdbvGP,series,804,2020-03-03 15:03:54,2020-03-03 15:04:10,play
3,03696,1TwKjpRDQyDn0rHp2EFadYovzfJ,series,0,2020-03-23 11:38:37,2020-03-23 11:38:37,click
4,22920,1TwL1JPis0Awx6Kk6kGhvm9BrV6,movie,0,2020-03-21 10:35:21,2020-03-21 10:35:21,click
...,...,...,...,...,...,...,...
36932499,17290,1eGHjeIefvU5QnVshsW43rh8jAt,OTD,8820,2020-07-10 17:44:00,2020-07-10 22:25:58,play
36932500,17290,1eGHji6OWS1fS2t40QmKaXIfZTT,OTD,3600,2020-07-10 15:59:00,2020-07-10 22:25:58,play
36932501,17290,1eGHjnk6hlJCKluUpN9u60JfoPt,OTD,581,2020-07-10 22:00:00,2020-07-10 22:25:58,play
36932502,17290,1eGHkFEqg1ZVddCe9aLkdkL28Ry,OTD,2700,2020-07-10 16:59:00,2020-07-10 22:25:58,play


In [5]:
df.nsmallest(3, 'client_upload_timestamp')

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
9888098,08979,1TwKjUy5etvVVXBFVMiNlLGzA4z,series,21,2019-11-11 11:57:02,2019-12-16 07:22:31,play
153684,18327,1TwL16LnVwOsNWT4bRytl5IZ8ua,movie,38,2019-11-12 01:58:23,2019-12-27 02:31:48,play
22032334,28234,1TwMrGfNJGRbLTZPXql7QadyJIx,JCM,803,2019-11-28 08:46:37,2019-11-30 15:27:47,play


In [6]:
df.nlargest(3, 'client_upload_timestamp')

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
12317518,29217,1ectAFNooRlwFnefUIQjKiVssHJ,JCM,4800,2020-08-12 13:00:00,2020-07-12 14:48:33,reserve
12441617,16664,1TwLHp5iT1l7C69BtlkZZWJ87B6,JCM,3300,2020-08-12 13:00:00,2020-07-12 13:55:32,reserve
10881559,02546,1TwLHigWmupQ0pPmZgfIMfp8adZ,JCM,3600,2020-08-12 12:30:00,2020-07-12 10:35:10,reserve


In [7]:
## check how many type of interactions are there 
df['interaction'].unique()

array(['play', 'click', 'purchase', 'search', 'reserve', 'record'],
      dtype=object)

In [8]:
## check how many type of item type are there
df['item_type'].unique()

array(['movie', 'series', 'JCM', 'OTD', 'BSD'], dtype=object)

In [9]:
print('process item meta...')
with open('/home/wtchiang/ADS_crossDomainRec/raw_data/item_metadata_v6_20200724_train.json', 'r') as j_file:
    item_meta = json.load(j_file)

item_meta_df = pd.DataFrame(item_meta)
vod_item_meta_df = item_meta_df[(item_meta_df['type'] == 'movie') | (item_meta_df['type'] == 'series')]
tv_item_meta_df = item_meta_df[~((item_meta_df['type'] == 'movie') | (item_meta_df['type'] == 'series'))]

process item meta...


In [10]:
filtered_vod_item = vod_item_meta_df[~((vod_item_meta_df["rating"] == "R-18") | (vod_item_meta_df["rating"] == "R-18E") | (vod_item_meta_df["rating"] == "R-20"))]

In [11]:
filtered_vod_item["item_id"]

8103      1TwKzJfSxxqXVW0tEE2hMCFSEfd
8104      1TwL0XY3ZLn6vuqjiG4iTbAz3zo
8106      1UOSDr6zGvwLZ1grj9wxUSQ48HI
8107      1TwKzI43uUlCgB3GZdkvGMBj5bv
8108      1TwKzVVYSDoOCLnWN8X9gAEllRX
                     ...             
978535    1TwKj4epzkrObuKYbWWalsKhHrK
978536    1TwKjhW7vZQWaTXYGJFpfMH5j7x
978537    1TwKjek2YyNQ1kcPjPpsALNR2P5
978538    1TwKjpmJl6o1pf4EU8cOfZ54PQD
978539    1TwKju1ZkCl73hgtNvvIhUvzwbP
Name: item_id, Length: 38266, dtype: object

Preprocess

In [48]:
# last k vod logs of each user
# last_k_vod = 20

In [12]:
vod_df = df[ (df['item_type']=='movie') | (df['item_type']=='series') ]

In [11]:
vod_df['item_id'].unique()

array(['1TwKybXi09qknpFn3DClPOOVCAK', '1TwKyRGvhmV61mO2uokTMFgNBup',
       '1TwKjd28kvL8VC151WbaKcdbvGP', ..., '1UTAKjow8Ctj4Tgb3nhmYVxwwr3',
       '1TwKzFGzV7E75QFtMwSo86XKAHf', '1ZFXRsXA6buYnvAuXqAhf0Sq5AS'],
      dtype=object)

In [13]:
common_user = set(df_test['user_id'].unique()) & set(vod_df[vod_df['interaction']=='play']['user_id'].unique())

In [14]:
len(common_user)

19959

In [11]:
df_test_click

,user_id,item_id
0,26832,1TwKis5ABTTgUMB36Lek9bbDRMd
1,02683,1TwKis5ABTTgUMB36Lek9bbDRMd
2,00789,1TwKis5ABTTgUMB36Lek9bbDRMd
3,23174,1TwKis5ABTTgUMB36Lek9bbDRMd
4,12552,1TwKis5ABTTgUMB36Lek9bbDRMd
...,...,...
101574,25973,1dZsgs549fkHqOzaQuLM4at7U4r
101575,07607,1dZsgs549fkHqOzaQuLM4at7U4r
101576,15987,1dZsgs549fkHqOzaQuLM4at7U4r
101577,28748,1dZsgs549fkHqOzaQuLM4at7U4r


In [15]:
# test_df = df_test[ (df_test['user_id'].isin(common_user)) & (df_test['item_id'].isin(filtered_vod_item["item_id"])) ]
train_df = vod_df[ (vod_df['user_id'].isin(common_user)) & (vod_df['item_id'].isin(filtered_vod_item["item_id"]))]
click_test_df = df_test_click[ (df_test_click['user_id'].isin(common_user)) & (df_test_click['item_id'].isin(filtered_vod_item['item_id'])) ]
purchase_test_df = df_test_purchase[(df_test_purchase['user_id'].isin(common_user)) & (df_test_purchase['item_id'].isin(filtered_vod_item['item_id']))]
search_test_df = df_test_search[(df_test_search['user_id'].isin(common_user)) & (df_test_search['item_id'].isin(filtered_vod_item['item_id']))] 

In [8]:
train_df

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
0,01640,1TwKybXi09qknpFn3DClPOOVCAK,movie,44,2020-03-17 10:34:33,2020-03-17 10:34:36,play
3,03696,1TwKjpRDQyDn0rHp2EFadYovzfJ,series,0,2020-03-23 11:38:37,2020-03-23 11:38:37,click
4,22920,1TwL1JPis0Awx6Kk6kGhvm9BrV6,movie,0,2020-03-21 10:35:21,2020-03-21 10:35:21,click
6,03827,1TwKinR47luf7vgUYpvCcgS3dSe,series,3857,2020-03-11 06:08:15,2020-03-11 06:08:45,play
8,02835,1TwKjo7hJFKDdVwghqDtcBMhxpo,series,0,2020-03-11 05:32:36,2020-03-11 05:32:35,click
...,...,...,...,...,...,...,...
10139695,05383,1TwKimaoPx4ZlzQpBeyKuRi28Gw,series,4,2020-06-03 23:59:00,2020-06-03 23:59:05,play
10139696,02379,1aCqOxZf8V83aEcKO9u87pG8rbi,series,0,2020-06-03 23:59:40,2020-06-03 23:59:40,click
10139698,00923,1TwKjFC0ctChfSTVwmIcMb4HAio,series,0,2020-03-27 00:00:59,2020-03-27 00:01:00,click
10139699,29662,1bAAa4p6dfIDM1e5rnMFO3N1Cfu,series,219,2020-06-21 13:51:50,2020-06-23 12:43:03,play


In [24]:
## store train and test data set
train_df.to_pickle("/home/syliu/AIR/Data/kk_Data/kk_vod_train_filtered.pkl")
test_df.to_pickle("/home/syliu/AIR/Data/kk_Data/kk_vod_test_filtered.pkl")


In [2]:
train_df = pd.read_pickle('/home/syliu/AIR/Data/kk_Data/kk_vod_train_filtered.pkl')
test_df = pd.read_pickle('/home/syliu/AIR/Data/kk_Data/kk_vod_test_filtered.pkl')

In [4]:
train_df['interaction'].unique()

array(['play', 'click', 'purchase', 'search'], dtype=object)

In [3]:
train_df

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
0,01640,1TwKybXi09qknpFn3DClPOOVCAK,movie,44,2020-03-17 10:34:33,2020-03-17 10:34:36,play
3,03696,1TwKjpRDQyDn0rHp2EFadYovzfJ,series,0,2020-03-23 11:38:37,2020-03-23 11:38:37,click
4,22920,1TwL1JPis0Awx6Kk6kGhvm9BrV6,movie,0,2020-03-21 10:35:21,2020-03-21 10:35:21,click
6,03827,1TwKinR47luf7vgUYpvCcgS3dSe,series,3857,2020-03-11 06:08:15,2020-03-11 06:08:45,play
8,02835,1TwKjo7hJFKDdVwghqDtcBMhxpo,series,0,2020-03-11 05:32:36,2020-03-11 05:32:35,click
...,...,...,...,...,...,...,...
10139695,05383,1TwKimaoPx4ZlzQpBeyKuRi28Gw,series,4,2020-06-03 23:59:00,2020-06-03 23:59:05,play
10139696,02379,1aCqOxZf8V83aEcKO9u87pG8rbi,series,0,2020-06-03 23:59:40,2020-06-03 23:59:40,click
10139698,00923,1TwKjFC0ctChfSTVwmIcMb4HAio,series,0,2020-03-27 00:00:59,2020-03-27 00:01:00,click
10139699,29662,1bAAa4p6dfIDM1e5rnMFO3N1Cfu,series,219,2020-06-21 13:51:50,2020-06-23 12:43:03,play


In [3]:
train_df['user_id'] = train_df['user_id'].apply(lambda x: f"user_{x}")
train_df['item_id'] = train_df['item_id'].apply(lambda x: f"vod_{x}")
# _train_graph = train_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)) 
# _train_graph = train_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*1.) 

In [6]:
x = train_df.groupby(['user_id', 'item_id','interaction']).size().to_frame(name = 'count').reset_index()

In [8]:
x['interaction'].replace(['play','click','search','purchase'],[5,4,3,2],inplace=True)

In [10]:
y = x[['user_id','item_id','interaction']]

In [15]:
y_interaction_2 = y[y['interaction'] == 2]
y_interaction_3 = y[y['interaction'] == 3]
y_interaction_4 = y[y['interaction'] == 4]
y_interaction_5 = y[y['interaction'] == 5]

In [14]:
pd.concat([y]).to_csv('/home/syliu/AIR/Data/kk_Data/kk_vod_rating_format.train',index=False, header=False, sep='\t')



In [ ]:
pd.concat([y_interaction_2]).to_csv('/home/syliu/AIR/Data/kk_Data/kk_vod_rating_format_purchase.train',index=False, header=False, sep='\t')
pd.concat([y_interaction_3]).to_csv('/home/syliu/AIR/Data/kk_Data/kk_vod_rating_format_search.train',index=False, header=False, sep='\t')
pd.concat([y_interaction_4]).to_csv('/home/syliu/AIR/Data/kk_Data/kk_vod_rating_format_click.train',index=False, header=False, sep='\t')
pd.concat([y_interaction_5]).to_csv('/home/syliu/AIR/Data/kk_Data/kk_vod_rating_format_play.train',index=False, header=False, sep='\t')

In [5]:
test_df

,user_id,item_id
0,12295,1TwKjkOad0UvYLOB8IJkhWROm95
1,27268,1TwKjFC0ctChfSTVwmIcMb4HAio
2,12034,1dteQlb5sxloVB4CRr9IT6t08bR
3,01458,1TwKimaoPx4ZlzQpBeyKuRi28Gw
4,22243,1VX4nwMAJbXO0z5bg2AWgLMXekx
...,...,...
81105,28993,1TwKymjFbb7ik14yfk36f4yrqXF
81106,05201,1TwKyqYeKX1mfYu3rb3lxDMIGAy
81107,13529,1TwKySaPYMozcWnTw4415Nv0hSi
81108,20983,1TwKjO3BST73olcrMpxp7RKT5dA


In [9]:
filtered_test = test_df[test_df['item_id'].isin(train_df['item_id'].unique())]

In [10]:
x = filtered_test[filtered_test['user_id'].isin(train_df['user_id'].unique())]

In [11]:
x

,user_id,item_id
0,12295,1TwKjkOad0UvYLOB8IJkhWROm95
1,27268,1TwKjFC0ctChfSTVwmIcMb4HAio
2,12034,1dteQlb5sxloVB4CRr9IT6t08bR
3,01458,1TwKimaoPx4ZlzQpBeyKuRi28Gw
4,22243,1VX4nwMAJbXO0z5bg2AWgLMXekx
...,...,...
81105,28993,1TwKymjFbb7ik14yfk36f4yrqXF
81106,05201,1TwKyqYeKX1mfYu3rb3lxDMIGAy
81107,13529,1TwKySaPYMozcWnTw4415Nv0hSi
81108,20983,1TwKjO3BST73olcrMpxp7RKT5dA


In [12]:
filtered_test_click = df_test_click[df_test_click['item_id'].isin(train_df['item_id'].unique())]
filtered_test_click = filtered_test_click[filtered_test_click['user_id'].isin(train_df['user_id'].unique())]

,user_id,item_id
0,26832,1TwKis5ABTTgUMB36Lek9bbDRMd
1,02683,1TwKis5ABTTgUMB36Lek9bbDRMd
2,00789,1TwKis5ABTTgUMB36Lek9bbDRMd
3,23174,1TwKis5ABTTgUMB36Lek9bbDRMd
4,12552,1TwKis5ABTTgUMB36Lek9bbDRMd
...,...,...
101574,25973,1dZsgs549fkHqOzaQuLM4at7U4r
101575,07607,1dZsgs549fkHqOzaQuLM4at7U4r
101576,15987,1dZsgs549fkHqOzaQuLM4at7U4r
101577,28748,1dZsgs549fkHqOzaQuLM4at7U4r


In [13]:
filtered_test_click = df_test_click[df_test_click['item_id'].isin(train_df['item_id'].unique())]
filtered_test_click = filtered_test_click[filtered_test_click['user_id'].isin(train_df['user_id'].unique())]

In [15]:
filtered_test_purchase = df_test_purchase[df_test_purchase['item_id'].isin(train_df['item_id'].unique())]
filtered_test_purchase = filtered_test_purchase[filtered_test_purchase['user_id'].isin(train_df['user_id'].unique())]

In [16]:
filtered_test_search = df_test_search[df_test_search['item_id'].isin(train_df['item_id'].unique())]
filtered_test_search = filtered_test_search[filtered_test_search['user_id'].isin(train_df['user_id'].unique())]

In [14]:
filtered_test_click

,user_id,item_id
1,02683,1TwKis5ABTTgUMB36Lek9bbDRMd
2,00789,1TwKis5ABTTgUMB36Lek9bbDRMd
3,23174,1TwKis5ABTTgUMB36Lek9bbDRMd
4,12552,1TwKis5ABTTgUMB36Lek9bbDRMd
5,21571,1TwKjEdIfaGAVaCFbPXbBkjRq55
...,...,...
101574,25973,1dZsgs549fkHqOzaQuLM4at7U4r
101575,07607,1dZsgs549fkHqOzaQuLM4at7U4r
101576,15987,1dZsgs549fkHqOzaQuLM4at7U4r
101577,28748,1dZsgs549fkHqOzaQuLM4at7U4r


In [17]:
test_df

,user_id,item_id
0,12295,1TwKjkOad0UvYLOB8IJkhWROm95
1,27268,1TwKjFC0ctChfSTVwmIcMb4HAio
2,12034,1dteQlb5sxloVB4CRr9IT6t08bR
3,01458,1TwKimaoPx4ZlzQpBeyKuRi28Gw
4,22243,1VX4nwMAJbXO0z5bg2AWgLMXekx
...,...,...
81105,28993,1TwKymjFbb7ik14yfk36f4yrqXF
81106,05201,1TwKyqYeKX1mfYu3rb3lxDMIGAy
81107,13529,1TwKySaPYMozcWnTw4415Nv0hSi
81108,20983,1TwKjO3BST73olcrMpxp7RKT5dA


In [18]:
filtered_test_search

,user_id,item_id
0,21915,1TwKjgtAF8kgbgVVKXlFC1yjTMm
1,17357,1TwKjgtAF8kgbgVVKXlFC1yjTMm
3,25558,1TwKyNsPtVpPVZIgzekANhIAljq
4,21629,1TwKysZYsYy1xt9h3S8fFJzKiqs
5,21604,1TwKyySBFnc5FQRX1FejD2sUWly
...,...,...
4437,02539,1TwKyqUTGG9qvQ9AmUJUj7r1hu5
4438,18108,1TwKzW2qCIV7AtfBjv89MRaNVrx
4439,06745,1TwKzW2qCIV7AtfBjv89MRaNVrx
4440,08274,1TwKzW2qCIV7AtfBjv89MRaNVrx


In [19]:
filtered_test_purchase

,user_id,item_id
0,06417,1TwKyNsPtVpPVZIgzekANhIAljq
1,25171,1TwKykoWlPxwdywoHKRXoKFeZ5X
2,09128,1TwKyosIZTMF77dESP3qeVdTXae
3,06015,1TwKyqods9OCjG0V3dMLpX9syIX
4,24484,1TwL0aqqlNTx59lgLOQ3LJ4wl7z
...,...,...
281,19296,1TwKyLyf0VBd34ddLSIKyRiU4kE
282,12570,1TwKyLyf0VBd34ddLSIKyRiU4kE
283,29201,1TwKyQKTMvIZOcpO6dLzmHcl7VH
284,02082,1TwKyQKTMvIZOcpO6dLzmHcl7VH


In [ ]:
filtered_test = test_df[test_df['item_id'].isin(train_df['item_id'].unique())]

In [22]:
filtered_test

,user_id,item_id
0,12295,1TwKjkOad0UvYLOB8IJkhWROm95
1,27268,1TwKjFC0ctChfSTVwmIcMb4HAio
2,12034,1dteQlb5sxloVB4CRr9IT6t08bR
3,01458,1TwKimaoPx4ZlzQpBeyKuRi28Gw
4,22243,1VX4nwMAJbXO0z5bg2AWgLMXekx
...,...,...
81105,28993,1TwKymjFbb7ik14yfk36f4yrqXF
81106,05201,1TwKyqYeKX1mfYu3rb3lxDMIGAy
81107,13529,1TwKySaPYMozcWnTw4415Nv0hSi
81108,20983,1TwKjO3BST73olcrMpxp7RKT5dA


In [3]:
# array(['play', 'click', 'purchase', 'search', 'reserve', 'record'], but vod type item dosen't have 'reserve', 'record'
train_df = train_df.reset_index(drop=True)
play_df = train_df[ train_df['interaction']=='play' ].reset_index(drop=True)
click_df = train_df[ train_df['interaction']=='click' ].reset_index(drop=True)
purchase_df = train_df[ train_df['interaction']=='purchase' ].reset_index(drop=True)
search_df = train_df[ train_df['interaction']=='search' ].reset_index(drop=True)
# play_test_df = x.reset_index(drop=True)
# click_test_df = filtered_test_click.reset_index(drop=True)
# purchase_test_df = filtered_test_purchase.reset_index(drop=True)
# search_test_df = filtered_test_search.reset_index(drop=True)
# all_df = pd.concat([play_test_df,click_test_df]).reset_index(drop=True)



In [4]:
play_df

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
0,01640,1TwKybXi09qknpFn3DClPOOVCAK,movie,44,2020-03-17 10:34:33,2020-03-17 10:34:36,play
1,03827,1TwKinR47luf7vgUYpvCcgS3dSe,series,3857,2020-03-11 06:08:15,2020-03-11 06:08:45,play
2,02161,1TwKj790BSuHnDzQkE8X3VOJpzt,series,70,2020-03-04 06:26:15,2020-03-04 06:26:24,play
3,08492,1TwKiKYku6RFnP9zAtow5gkiTIj,series,1059,2020-03-17 09:45:58,2020-03-17 09:46:20,play
4,11084,1TwKjDtsIKwNR0XiGICC3kKI3fu,series,1493,2020-03-11 09:43:16,2020-03-11 09:43:47,play
...,...,...,...,...,...,...,...
4214344,04034,1TwKj9fQWFSU3bxaTVDkZ5F8gIK,series,5,2020-04-04 23:59:05,2020-04-04 23:59:12,play
4214345,04250,1XNiIPE65pkA4YXKzJI3wtoZAxp,series,677,2020-05-19 15:37:45,2020-05-20 15:52:03,play
4214346,05383,1TwKimaoPx4ZlzQpBeyKuRi28Gw,series,4,2020-06-03 23:59:00,2020-06-03 23:59:05,play
4214347,29662,1bAAa4p6dfIDM1e5rnMFO3N1Cfu,series,219,2020-06-21 13:51:50,2020-06-23 12:43:03,play


In [7]:
train_df

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
0,01640,1TwKybXi09qknpFn3DClPOOVCAK,movie,44,2020-03-17 10:34:33,2020-03-17 10:34:36,play
1,03696,1TwKjpRDQyDn0rHp2EFadYovzfJ,series,0,2020-03-23 11:38:37,2020-03-23 11:38:37,click
2,22920,1TwL1JPis0Awx6Kk6kGhvm9BrV6,movie,0,2020-03-21 10:35:21,2020-03-21 10:35:21,click
3,03827,1TwKinR47luf7vgUYpvCcgS3dSe,series,3857,2020-03-11 06:08:15,2020-03-11 06:08:45,play
4,02835,1TwKjo7hJFKDdVwghqDtcBMhxpo,series,0,2020-03-11 05:32:36,2020-03-11 05:32:35,click
...,...,...,...,...,...,...,...
6581730,05383,1TwKimaoPx4ZlzQpBeyKuRi28Gw,series,4,2020-06-03 23:59:00,2020-06-03 23:59:05,play
6581731,02379,1aCqOxZf8V83aEcKO9u87pG8rbi,series,0,2020-06-03 23:59:40,2020-06-03 23:59:40,click
6581732,00923,1TwKjFC0ctChfSTVwmIcMb4HAio,series,0,2020-03-27 00:00:59,2020-03-27 00:01:00,click
6581733,29662,1bAAa4p6dfIDM1e5rnMFO3N1Cfu,series,219,2020-06-21 13:51:50,2020-06-23 12:43:03,play


In [49]:
train_df['user_id'] = train_df['user_id'].apply(lambda x: f"user_{x}")
train_df['item_id'] = train_df['item_id'].apply(lambda x: f"vod_{x}")
# _train_graph = train_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)) 
_train_graph = train_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*1.) 

In [5]:
play_df['user_id'] = play_df['user_id'].apply(lambda x: f"user_{x}")
play_df['item_id'] = play_df['item_id'].apply(lambda x: f"vod_{x}")
# _play_graph = play_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)) 
_play_graph = play_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*1.) 

In [6]:
click_df['user_id'] = click_df['user_id'].apply(lambda x: f"user_{x}")
click_df['item_id'] = click_df['item_id'].apply(lambda x: f"vod_{x}")
# pure six months vod click
_click_graph = click_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)) 

In [7]:
purchase_df['user_id'] = purchase_df['user_id'].apply(lambda x: f"user_{x}")
purchase_df['item_id'] = purchase_df['item_id'].apply(lambda x: f"vod_{x}")
# pure six months vod purchase
_purchase_graph = purchase_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)) 

In [8]:
search_df['user_id'] = search_df['user_id'].apply(lambda x: f"user_{x}")
search_df['item_id'] = search_df['item_id'].apply(lambda x: f"vod_{x}")
# pure six months vod search
_search_graph = search_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)) 

In [33]:
play_test_df['user_id'] = play_test_df['user_id'].apply(lambda x: f"user_{x}")
play_test_df['item_id'] = play_test_df['item_id'].apply(lambda x: f"vod_{x}")

_play_test_graph = play_test_df.groupby(['user_id', 'item_id']).size()

In [52]:
# click_test_df['user_id'] = click_test_df['user_id'].apply(lambda x: f"user_{x}")
# click_test_df['item_id'] = click_test_df['item_id'].apply(lambda x: f"vod_{x}")

# _click_test_graph = click_test_df.groupby(['user_id', 'item_id']).size()

# purchase_test_df['user_id'] = purchase_test_df['user_id'].apply(lambda x: f"user_{x}")
# purchase_test_df['item_id'] = purchase_test_df['item_id'].apply(lambda x: f"vod_{x}")

# _purchase_test_graph = purchase_test_df.groupby(['user_id', 'item_id']).size()

# search_test_df['user_id'] = search_test_df['user_id'].apply(lambda x: f"user_{x}")
# search_test_df['item_id'] = search_test_df['item_id'].apply(lambda x: f"vod_{x}")

# _search_test_graph = search_test_df.groupby(['user_id', 'item_id']).size()

all_df['user_id'] = all_df['user_id'].apply(lambda x: f"user_{x}")
all_df['item_id'] = all_df['item_id'].apply(lambda x: f"vod_{x}")

_all_test_graph = all_df.groupby(['user_id', 'item_id']).size()

In [50]:
_train_graph

user_id     item_id                        
user_00002  vod_1TwKib06tX8FGeY9KZlCklG6EFr    0.693147
            vod_1TwKifeBdYT14ruFjuOpqpLRtKH    2.708050
            vod_1TwKinR47luf7vgUYpvCcgS3dSe    1.386294
            vod_1TwKjCouqBbjDuHwY2Uj6P3VGM9    2.995732
            vod_1TwKjJVd4eBcs2VJU2deTI9Rw6g    1.945910
                                                 ...   
user_32412  vod_1dRP5pcza0ZcnJzLfctyOjkNXdJ    0.693147
user_32413  vod_1TwKj3l2tN7svekHAFVCxD38o2o    0.693147
            vod_1TwKyNaToFXB9OkBXWTO41jtiBj    1.098612
            vod_1TwKyVbvpcUwVLRFtN28Xq4c4jK    1.386294
            vod_1WUroViYVKtqIU6vEbljPQYlb6A    1.386294
Length: 1199817, dtype: float64

In [11]:
_purchase_graph

user_id     item_id                        
user_00008  vod_1TwKijtHn8kAotGhaphBZOxveKc    0.693147
            vod_1TwKio5ZgXefyPyEtmK2NG63oZy    1.386294
            vod_1TwKipQj5FqP8rtJxGBihJqCq0W    0.693147
            vod_1TwKirGRs4xZ25MmqOZt6LN9xg6    0.693147
            vod_1TwKirvxEahRhTnVMstr8f9Ky8i    1.098612
                                                 ...   
user_32409  vod_1cWudtJJAdKo4JJKywO0JUPLHnA    0.693147
user_32410  vod_1TwKinYIxwnX54FxtjUlYXXuWmJ    0.693147
            vod_1TwKyRscOym7LobWP2NzIE2urbi    0.693147
            vod_1TwKyUG99I9kLaebjxouXIOHJuW    0.693147
            vod_1UI6SwguyITSvfXbqG85ScQ8vkQ    0.693147
Length: 57660, dtype: float64

In [16]:
_click_test_graph

user_id     item_id                        
user_00004  vod_1TwKiyCqEpx5w2xCa85plFPvZAf    1
            vod_1TwKjJYZtR5DdyViVSgr2EehAZ5    1
            vod_1TwKjOqnNJxjp1g1ijMnvCIIce5    1
            vod_1TwKjSY35or77TFD4Y2qh63QJjA    1
            vod_1TwKjTERLsa5UFSDfcchfxywkeH    1
                                              ..
user_32412  vod_1TwKyZo7Ius0CRs1qfo5v7unzU3    1
            vod_1TwKycW62k8Syxu8TapnkELNOtN    1
            vod_1TwL0XgPKZqnKDiWvgWxqaQS1lQ    1
            vod_1TwL0b3CSNJxBFOXcHg3dtnjIxR    1
user_32413  vod_1TwKyqYeKX1mfYu3rb3lxDMIGAy    1
Length: 87329, dtype: int64

In [55]:
_all_test_graph

user_id     item_id                        
user_00002  vod_1dteQnSRcHZfiDlEp05fXOoxi6H    1
user_00004  vod_1TwKiyCqEpx5w2xCa85plFPvZAf    1
            vod_1TwKjJYZtR5DdyViVSgr2EehAZ5    1
            vod_1TwKjOqnNJxjp1g1ijMnvCIIce5    1
            vod_1TwKjSY35or77TFD4Y2qh63QJjA    1
                                              ..
user_32413  vod_1TwKyqYeKX1mfYu3rb3lxDMIGAy    1
            vod_1bCztLTwD4D53wvoXT1Ki3xQl7W    1
            vod_1cWudlaxyle9OSXeaGIj6Jc8KXF    1
            vod_1dRP5pcza0ZcnJzLfctyOjkNXdJ    1
            vod_1dteQrdb2OICye62Kmp83sSTIwF    1
Length: 147920, dtype: int64

In [38]:
_df = pd.concat([_train_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_filtered.train', header=False, sep='\t') 

In [39]:
_df = pd.concat([_play_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_play_filtered.train', header=False, sep='\t') 

In [40]:
_df = pd.concat([_click_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_click_filtered.txt', header=False, sep='\t') 

In [41]:
_df = pd.concat([_purchase_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_purchase_filtered.txt', header=False, sep='\t') 

In [42]:
_df = pd.concat([_search_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_search_filtered.txt', header=False, sep='\t') 

In [43]:
_df = pd.concat([_test_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_filtered.test', header=False, sep='\t') 

In [56]:
# _df = pd.concat([_train_graph])
# _df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_filtered.train', header=False, sep='\
_df = pd.concat([_play_test_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_play_filtered.test', header=False, sep='\t') 
_df = pd.concat([_click_test_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_click_filtered.test', header=False, sep='\t') 
_df = pd.concat([_all_test_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_different_interaction_input/kk_vod_all_filtered.test', header=False, sep='\t') 

In [17]:
_df = pd.concat([_click_test_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_input/six_months_vod_click_filtered.test', header=False, sep='\t') 

In [18]:
_df = pd.concat([_purchase_test_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_input/six_months_vod_purchase_filtered.test', header=False, sep='\t') 

In [19]:
_df = pd.concat([_search_test_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_input/six_months_vod_search_filtered.test', header=False, sep='\t') 

In [107]:
# # test
# df_test['user_id'] = df_test['user_id'].apply(lambda x: f"user_{x}")
# df_test['item_id'] = df_test['item_id'].apply(lambda x: f"vod_{x}")
# test_graph = df_test.groupby(['user_id', 'item_id']).size()
# test_graph.to_csv('/home/syliu/AIR/Data/kk_Data/air_input/test_ui.txt', header=False, sep='\t')

Air kk ?? inputs

In [18]:
play_df.head(10)

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
0,user_01640,vod_1TwKybXi09qknpFn3DClPOOVCAK,movie,44,2020-03-17 10:34:33,2020-03-17 10:34:36,play
1,user_03827,vod_1TwKinR47luf7vgUYpvCcgS3dSe,series,3857,2020-03-11 06:08:15,2020-03-11 06:08:45,play
2,user_02161,vod_1TwKj790BSuHnDzQkE8X3VOJpzt,series,70,2020-03-04 06:26:15,2020-03-04 06:26:24,play
3,user_08492,vod_1TwKiKYku6RFnP9zAtow5gkiTIj,series,1059,2020-03-17 09:45:58,2020-03-17 09:46:20,play
4,user_11084,vod_1TwKjDtsIKwNR0XiGICC3kKI3fu,series,1493,2020-03-11 09:43:16,2020-03-11 09:43:47,play
5,user_00250,vod_1TwKjFC0ctChfSTVwmIcMb4HAio,series,1429,2020-03-20 08:11:57,2020-03-20 08:12:27,play
6,user_24380,vod_1TwKj4hsI5lcouZyKsnGk9SJyJH,series,83,2020-03-17 08:45:35,2020-03-17 08:45:35,play
7,user_15699,vod_1TwKjUOByXdbicP08Xd542HQeWJ,series,3893,2020-03-03 13:46:50,2020-03-03 13:47:20,play
8,user_08708,vod_1TwKjRaQn7b72dkH7NoV0aLuFz9,series,47,2020-03-11 07:28:46,2020-03-11 07:28:47,play
9,user_07014,vod_1TwKirwfJjBWRLZ8VlN3XuXc768,series,1412,2020-03-21 11:46:39,2020-03-21 11:47:05,play


In [39]:
play_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*10.)

user_id     item_id                        
user_00002  vod_1TwKifeBdYT14ruFjuOpqpLRtKH    26.390573
            vod_1TwKinR47luf7vgUYpvCcgS3dSe    10.986123
            vod_1TwKjCouqBbjDuHwY2Uj6P3VGM9    29.444390
            vod_1TwKjJVd4eBcs2VJU2deTI9Rw6g    16.094379
            vod_1TwKjKMVWc4G44Wz4IjY4awjBEG    10.986123
                                                 ...    
user_32412  vod_1aLKAwr5RA8GsP8CwCOeeeXiMUC    10.986123
            vod_1dRP5pcza0ZcnJzLfctyOjkNXdJ     6.931472
user_32413  vod_1TwKyNaToFXB9OkBXWTO41jtiBj     6.931472
            vod_1TwKyVbvpcUwVLRFtN28Xq4c4jK     6.931472
            vod_1WUroViYVKtqIU6vEbljPQYlb6A     6.931472
Length: 531275, dtype: float64

In [28]:
play_df['play_amount_second'].mean()

1277.9135823824747

In [29]:
play_df['play_amount_second'].std()

1505.0394637260704

Air kk time inputs 

In [15]:
last_k_vod = 20 # recent 20 as positive and rest treated as not relevant

In [19]:
play_last_df = play_df.sort_values('client_upload_timestamp').groupby('user_id').apply(lambda x: x.tail(last_k_vod)).reset_index(drop=True)

In [21]:
play_irrelevant_df = play_df.sort_values('client_upload_timestamp').groupby('user_id').apply(lambda x: x.head(len(x)-last_k_vod)).reset_index(drop=True)

In [28]:
len(play_last_df['user_id'].unique())

19844

In [29]:
len(play_irrelevant_df['user_id'].unique())

14837

In [26]:
play_positive_graph = play_last_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*10.)
play_negative_graph = play_irrelevant_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*10.)

In [30]:
play_positive_graph

user_id     item_id                        
user_00002  vod_1TwKjh0yBFmvudfCNI8FWWX7wZ4     6.931472
            vod_1TwKy8icygBIJOfIeLM2LU6xXhy    16.094379
            vod_1TwKyB2UEFf1pN3fHUgtMKBxTmb    10.986123
            vod_1TwKyGE2QKiOFpFmDhiTK8gO9jN     6.931472
            vod_1TwKyU9yzCk3CxeQ1MZ7jFRHzlU    13.862944
                                                 ...    
user_32411  vod_1cRGB4B1QEaF00uFhKgQ25pEKBy     6.931472
user_32412  vod_1TwKjZfIhthaWh2qQNwzLRK64tV    30.445224
user_32413  vod_1TwKyNaToFXB9OkBXWTO41jtiBj     6.931472
            vod_1TwKyVbvpcUwVLRFtN28Xq4c4jK     6.931472
            vod_1WUroViYVKtqIU6vEbljPQYlb6A     6.931472
Length: 112714, dtype: float64

In [31]:
pd.concat([play_positive_graph]).to_csv('/home/syliu/AIR/Data/kk_Data/air_neg_input/play_positive_time.txt', header=False, sep='\t')
pd.concat([play_negative_graph]).to_csv('/home/syliu/AIR/Data/kk_Data/air_neg_input/play_negative_time.txt', header=False, sep='\t')


Last k user log as input for LSTP

In [22]:
last_k_vod = 20 

In [23]:
play_df

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
0,user_01640,vod_1TwKybXi09qknpFn3DClPOOVCAK,movie,44,2020-03-17 10:34:33,2020-03-17 10:34:36,play
1,user_03827,vod_1TwKinR47luf7vgUYpvCcgS3dSe,series,3857,2020-03-11 06:08:15,2020-03-11 06:08:45,play
2,user_02161,vod_1TwKj790BSuHnDzQkE8X3VOJpzt,series,70,2020-03-04 06:26:15,2020-03-04 06:26:24,play
3,user_08492,vod_1TwKiKYku6RFnP9zAtow5gkiTIj,series,1059,2020-03-17 09:45:58,2020-03-17 09:46:20,play
4,user_11084,vod_1TwKjDtsIKwNR0XiGICC3kKI3fu,series,1493,2020-03-11 09:43:16,2020-03-11 09:43:47,play
...,...,...,...,...,...,...,...
4214344,user_04034,vod_1TwKj9fQWFSU3bxaTVDkZ5F8gIK,series,5,2020-04-04 23:59:05,2020-04-04 23:59:12,play
4214345,user_04250,vod_1XNiIPE65pkA4YXKzJI3wtoZAxp,series,677,2020-05-19 15:37:45,2020-05-20 15:52:03,play
4214346,user_05383,vod_1TwKimaoPx4ZlzQpBeyKuRi28Gw,series,4,2020-06-03 23:59:00,2020-06-03 23:59:05,play
4214347,user_29662,vod_1bAAa4p6dfIDM1e5rnMFO3N1Cfu,series,219,2020-06-21 13:51:50,2020-06-23 12:43:03,play


In [24]:
## if the number of element is smaller than last_k_vod, tail will group as many elements as possible
play_last_df = play_df.sort_values('client_upload_timestamp').groupby('user_id').apply(lambda x: x.tail(last_k_vod)).reset_index(drop=True)
# click_last_df = click_df.sort_values('client_upload_timestamp').groupby('user_id').apply(lambda x: x.tail(last_k_vod)).reset_index(drop=True)
# purchase_last_df = purchase_df.sort_values('client_upload_timestamp').groupby('user_id').apply(lambda x: x.tail(last_k_vod)).reset_index(drop=True)  
# search_last_df = search_df.sort_values('client_upload_timestamp').groupby('user_id').apply(lambda x: x.tail(last_k_vod)).reset_index(drop=True)
# reserve_last_df = reserve_df.sort_values('client_upload_timestamp').groupby('user_id').apply(lambda x: x.tail(last_k_vod)).reset_index(drop=True)
# record_last_df = record_df.sort_values('client_upload_timestamp').groupby('user_id').apply(lambda x: x.tail(last_k_vod)).reset_index(drop=True)

In [24]:
play_last_df

,user_id,item_id,item_type,play_amount_second,client_upload_timestamp,server_upload_timestamp,interaction
0,user_00002,vod_1ZFXALUE7J5d8MBIs0ivtww3cq9,series,642,2020-04-20 21:47:08,2020-04-20 21:47:08,play
1,user_00002,vod_1ZFXALUE7J5d8MBIs0ivtww3cq9,series,3045,2020-04-22 03:18:20,2020-04-22 03:18:52,play
2,user_00002,vod_1ZFXALUE7J5d8MBIs0ivtww3cq9,series,4257,2020-04-22 04:35:56,2020-04-22 04:36:08,play
3,user_00002,vod_1TwKyU9yzCk3CxeQ1MZ7jFRHzlU,movie,411,2020-04-22 04:43:05,2020-04-22 04:43:13,play
4,user_00002,vod_1TwKyU9yzCk3CxeQ1MZ7jFRHzlU,movie,3693,2020-04-22 11:11:34,2020-04-22 11:12:04,play
...,...,...,...,...,...,...,...
310373,user_32412,vod_1TwKjZfIhthaWh2qQNwzLRK64tV,series,1470,2020-07-12 14:16:28,2020-07-12 14:16:59,play
310374,user_32412,vod_1TwKjZfIhthaWh2qQNwzLRK64tV,series,1470,2020-07-12 14:41:03,2020-07-12 14:41:34,play
310375,user_32413,vod_1WUroViYVKtqIU6vEbljPQYlb6A,movie,60,2020-07-12 07:27:10,2020-07-12 07:27:21,play
310376,user_32413,vod_1TwKyVbvpcUwVLRFtN28Xq4c4jK,movie,2701,2020-07-12 08:13:06,2020-07-12 08:13:19,play


In [25]:
# play_last_df['user_id'] = play_last_df['user_id'].apply(lambda x: f"user_{x}")
# play_last_df['item_id'] = play_last_df['item_id'].apply(lambda x: f"vod_{x}")
play_last_graph = play_last_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*10.) # add weight here

# click_last_df['user_id'] = click_last_df['user_id'].apply(lambda x: f"user_{x}")
# click_last_df['item_id'] = click_last_df['item_id'].apply(lambda x: f"vod_{x}")
# click_last_graph = click_last_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*1.) # add weight here

# purchase_last_df['user_id'] = purchase_last_df['user_id'].apply(lambda x: f"user_{x}")
# purchase_last_df['item_id'] = purchase_last_df['item_id'].apply(lambda x: f"vod_{x}")
# purchase_last_graph = purchase_last_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*1.) # add weight here

# search_last_df['user_id'] = search_last_df['user_id'].apply(lambda x: f"user_{x}")
# search_last_df['item_id'] = search_last_df['item_id'].apply(lambda x: f"vod_{x}")
# search_last_graph = search_last_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*1.) # add weight here

# reserve_last_df['user_id'] = reserve_last_df['user_id'].apply(lambda x: f"user_{x}")
# reserve_last_df['item_id'] = reserve_last_df['item_id'].apply(lambda x: f"vod_{x}")
# reserve_last_graph = reserve_last_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*1.) # add weight here

# record_last_df['user_id'] = record_last_df['user_id'].apply(lambda x: f"user_{x}")
# record_last_df['item_id'] = reserve_last_df['item_id'].apply(lambda x: f"vod_{x}")
# record_last_graph = record_last_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)*1.)

In [32]:
play_df_lstp = play_df.copy(deep=True)
play_df_lstp['user_id'] = play_df_lstp['user_id'].apply(lambda x: x.replace("user_", "play_"))
play_df_lstp = play_df_lstp.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.)) 



In [33]:
play_df_lstp

user_id     item_id                        
play_00002  vod_1TwKifeBdYT14ruFjuOpqpLRtKH    2.639057
            vod_1TwKinR47luf7vgUYpvCcgS3dSe    1.098612
            vod_1TwKjCouqBbjDuHwY2Uj6P3VGM9    2.944439
            vod_1TwKjJVd4eBcs2VJU2deTI9Rw6g    1.609438
            vod_1TwKjKMVWc4G44Wz4IjY4awjBEG    1.098612
                                                 ...   
play_32412  vod_1aLKAwr5RA8GsP8CwCOeeeXiMUC    1.098612
            vod_1dRP5pcza0ZcnJzLfctyOjkNXdJ    0.693147
play_32413  vod_1TwKyNaToFXB9OkBXWTO41jtiBj    0.693147
            vod_1TwKyVbvpcUwVLRFtN28Xq4c4jK    0.693147
            vod_1WUroViYVKtqIU6vEbljPQYlb6A    0.693147
Length: 531275, dtype: float64

In [34]:
play_last_graph

user_id     item_id                        
user_00002  vod_1TwKjh0yBFmvudfCNI8FWWX7wZ4     6.931472
            vod_1TwKy8icygBIJOfIeLM2LU6xXhy    16.094379
            vod_1TwKyB2UEFf1pN3fHUgtMKBxTmb    10.986123
            vod_1TwKyGE2QKiOFpFmDhiTK8gO9jN     6.931472
            vod_1TwKyU9yzCk3CxeQ1MZ7jFRHzlU    13.862944
                                                 ...    
user_32411  vod_1cRGB4B1QEaF00uFhKgQ25pEKBy     6.931472
user_32412  vod_1TwKjZfIhthaWh2qQNwzLRK64tV    30.445224
user_32413  vod_1TwKyNaToFXB9OkBXWTO41jtiBj     6.931472
            vod_1TwKyVbvpcUwVLRFtN28Xq4c4jK     6.931472
            vod_1WUroViYVKtqIU6vEbljPQYlb6A     6.931472
Length: 112714, dtype: float64

In [36]:
pd.concat([play_df_lstp, play_last_graph]).to_csv('/home/syliu/AIR/Data/kk_Data/lstp_input/lstp_input.txt', header=False, sep='\t')
print('saved in ./input/lstp_input.txt')

saved in ./input/lstp_input.txt


Preprcoess input for CPR 

In [39]:
last_k_tv = 100

In [37]:
tv_df = df[ ~((df['item_type']=='movie') | (df['item_type']=='series')) ]

In [40]:
tv_df = tv_df[ tv_df['user_id'].isin(common_user) ]
tv_df = tv_df[ (tv_df['client_upload_timestamp'] > time_start) & (tv_df['client_upload_timestamp'] <= time_end) ]
tv_last_df = tv_df.sort_values('client_upload_timestamp').groupby('user_id').apply(lambda x: x.tail(last_k_tv)).reset_index(drop=True)

In [42]:
tv_last_df['user_id'] = tv_last_df['user_id'].apply(lambda x: f"user_{x}")
tv_last_df['item_id'] = tv_last_df['item_id'].apply(lambda x: f"tv_{x}")
tv_last_graph = tv_last_df.groupby(['user_id', 'item_id']).size().apply(lambda x: log(x+1.))

In [45]:
pd.concat([_play_graph,tv_last_graph]).to_csv('/home/syliu/AIR/Data/kk_Data/cpr_input/cpr_vod_tv.txt', header=False, sep='\t')


In [71]:
pd.concat([tv_last_graph]).to_csv('/home/syliu/AIR/Data/kk_Data/cpr_input/tv_last.txt', header=False, sep='\t')

Vod metadata preprocess

In [4]:

# print('process item meta...')
# with open('/home/wtchiang/ADS_crossDomainRec/raw_data/item_metadata_v6_20200724_train.json', 'r') as j_file:
#     item_meta = json.load(j_file)

# item_meta_df = pd.DataFrame(item_meta)
# vod_item_meta_df = item_meta_df[(item_meta_df['type'] == 'movie') | (item_meta_df['type'] == 'series')]
# tv_item_meta_df = item_meta_df[~((item_meta_df['type'] == 'movie') | (item_meta_df['type'] == 'series'))]

process item meta...


In [47]:
common_vod_id = set(vod_df['item_id'].unique()) & set(vod_item_meta_df['item_id'].unique())

# vod_item_meta_df[ (vod_item_meta_df['item_id'] == "1TwKzJfSxxqXVW0tEE2hMCFSEfd") ]

In [48]:
len(common_vod_id)

27659

In [49]:
filtered_vod_item["item_id"]

8103      1TwKzJfSxxqXVW0tEE2hMCFSEfd
8104      1TwL0XY3ZLn6vuqjiG4iTbAz3zo
8106      1UOSDr6zGvwLZ1grj9wxUSQ48HI
8107      1TwKzI43uUlCgB3GZdkvGMBj5bv
8108      1TwKzVVYSDoOCLnWN8X9gAEllRX
                     ...             
978535    1TwKj4epzkrObuKYbWWalsKhHrK
978536    1TwKjhW7vZQWaTXYGJFpfMH5j7x
978537    1TwKjek2YyNQ1kcPjPpsALNR2P5
978538    1TwKjpmJl6o1pf4EU8cOfZ54PQD
978539    1TwKju1ZkCl73hgtNvvIhUvzwbP
Name: item_id, Length: 38266, dtype: object

In [57]:
# common_filtered_vod_meta_df = vod_item_meta_df[ vod_item_meta_df['item_id'].isin(common_vod_id) and vod_item_meta_df['item_id'].isin(filtered_vod_item["item_id"]) ]
common_filtered_vod_meta_df = vod_item_meta_df[ vod_item_meta_df['item_id'].isin(filtered_vod_item["item_id"]) & vod_item_meta_df['item_id'].isin(common_vod_id) ]

In [59]:
common_filtered_vod_meta_df = common_filtered_vod_meta_df.reset_index(drop=True)

In [60]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [62]:
common_filtered_vod_meta_df.head(1)

,item_id,title,description,start_time,end_time,channel_id,genre,artist,type,available,rating,duration,mood,keyword
0,1TwL0XY3ZLn6vuqjiG4iTbAz3zo,太平洋の翼,日本海軍不滅の戦闘機”紫電改”の活躍を描くアクション大作！太平洋戦争末期、日本海軍不滅の戦闘...,1343746800,1630940399,0,"[J2:邦画, J209:時代劇･戦争]","[松林宗恵, 三船敏郎, 加山雄三, 夏木陽介, 佐藤允, 星由里子, 團伊玖磨, 円谷英二...",movie,True,G,6071.0,None,None


In [64]:
## seprate each entry with genre value as it is essentially a list 
common_filtered_vod_meta_df = explode(common_filtered_vod_meta_df, ['genre'])

In [65]:
common_filtered_vod_meta_df.head(1)

,artist,available,channel_id,description,duration,end_time,item_id,keyword,mood,rating,start_time,title,type,genre
0,"[松林宗恵, 三船敏郎, 加山雄三, 夏木陽介, 佐藤允, 星由里子, 團伊玖磨, 円谷英二...",True,0,日本海軍不滅の戦闘機”紫電改”の活躍を描くアクション大作！太平洋戦争末期、日本海軍不滅の戦闘...,6071.0,1630940399,1TwL0XY3ZLn6vuqjiG4iTbAz3zo,,,G,1343746800,太平洋の翼,movie,J2:邦画


In [66]:
common_filtered_vod_meta_df['item_id'] = common_filtered_vod_meta_df['item_id'].apply(lambda x: f"vod_{x}")

In [67]:
common_filtered_vod_meta_df.head(1)

,artist,available,channel_id,description,duration,end_time,item_id,keyword,mood,rating,start_time,title,type,genre
0,"[松林宗恵, 三船敏郎, 加山雄三, 夏木陽介, 佐藤允, 星由里子, 團伊玖磨, 円谷英二...",True,0,日本海軍不滅の戦闘機”紫電改”の活躍を描くアクション大作！太平洋戦争末期、日本海軍不滅の戦闘...,6071.0,1630940399,vod_1TwL0XY3ZLn6vuqjiG4iTbAz3zo,,,G,1343746800,太平洋の翼,movie,J2:邦画


In [68]:
# _genre_graph = common_vod_meta_df.groupby(['item_id', 'genre']).size().apply(lambda x: log(x+1.)) 
_genre_graph = common_filtered_vod_meta_df.groupby(['item_id', 'genre']).size()

In [69]:
_genre_graph

item_id                          genre         
vod_1TwKiI7OEYgY5ohXDlnUbg8LlvI  J407:ヒューマンドラマ     1
                                 J411:深夜ドラマ        1
                                 J4:国内ドラマ          1
vod_1TwKiIIINOcW8zGRTAhyuRVDBv7  J410:時代劇          1
                                 J4:国内ドラマ          1
                                                  ..
vod_1ecrHnPqjQ7eXA2aiUZLAQCMaWR  n2:ドラマ／エンタメ       1
vod_1ecrHo4UFa6kRty5YQpbEFSee4L  J613:趣味･教養        1
                                 J6:エンタメ･その他       1
                                 n101:ドキュメンタリー     1
                                 n1:ドキュメンタリー·教養    1
Length: 66367, dtype: int64

In [70]:
_df = pd.concat([_genre_graph])
_df.to_csv('/home/syliu/AIR/Data/kk_Data/air_input/six_months_vod_meta_data_filtered.txt', header=False, sep='\t') 